In [1]:
from sympy import *
import numpy as np
import copy
import math
import pandas as pd
import itertools
import time
import pylab as pl
from IPython import display


In [5]:
def pso(objective,poblacion,CogConst,SocialConst,Inertia={"min":.01 ,"max":1,"delta":0.95,"gen":10},
        iterations=1,case="minimo",figure=False):
  
  print('Inicializando algoritmo...')
  # selector de direccion 
  if case == "minimo":
    funcion = lambda obNew,obOld : obNew < obOld
    Gbest={"value":1000000000000000000000,"position":False}
  elif case == "maximo":
#     print("entro")
    funcion = lambda obNew,obOld : obNew >= obOld #Se había cambiado la condicion a >=
    Gbest={"value":-1000000000000000000000,"position":False}
  else:
    Gbest={"value":1000000000000000000000,"position":False}
    funcion = lambda obNew,obOld : obNew < obOld

  # incializacion de inercia 
  Wi=Inertia["max"]
  #inicializaciones de mejores locales
  for particle in poblacion:    
    perform = objective([particle["position"]])
    particle["Pbest"]={"value":perform,"position":particle["position"]}          
  
  GbestHist = list()
  print('Realizando optimizacion con PSO..')
  for i in range(iterations):
  #ajuste de mejores   
    for particle in poblacion:
      if 40<=particle["position"][0]<=90 and 0.1<=particle["position"][1]<=0.3 and 140<=particle["position"][2]<=200 and 0.4<=particle["position"][3]<=0.6: 
        perform = objective([particle["position"]])
      else: 
        perform = -1000000000000000000000
      if funcion(perform,particle["Pbest"]["value"]):
        particle["Pbest"]={"value":perform,"position":particle["position"]}
      if funcion(perform,Gbest["value"]):
        Gbest["value"]=perform
        Gbest["position"]=particle["position"]
        GbestHist.append(Gbest)

  

    for particle in poblacion:
      #generacion de aleatorios 
      R0=np.random.uniform()
      R1=np.random.uniform()
      R2=np.random.uniform()
      #velocity update
      #Componente Inercial
      #iner=list(map((Wi).__mul__, particle["speed"]))
      iner=np.array(particle["speed"])*Wi*R0
      #(iner)
      #Componente cognitivo
      cog=CogConst*R1*(np.array(particle["Pbest"]["position"])-np.array(particle["position"]))
      #print(cog)
      #Componente social
      social=SocialConst*R2*(np.array(Gbest["position"])-np.array(particle["position"]))
      #print(social)
#       print('Aporte de inercia',iner)
#       print('Aporte cognitivo',cog)
#       print('Aporte social',social)
      vel=iner+cog+social
      #print(vel)
      #print(particle["position"])
      pos = list(np.array(particle["position"]) + vel)
      
#       if 40<pos[0]<90 and 0.1<pos[1]<0.3 and 140<pos[2]<200 and 0.4<pos[3]<0.6:            
#         print(pos)
      particle["position"] = pos
      particle["speed"] = list(vel)
    #print(poblacion)
#     if figure:
#         solutionPrint(objective,figure[0], figure[1], poblacion)
    #ajustar inertia 
    if i % Inertia["gen"]==0 and Wi>Inertia["min"] :
      Wi=Wi*Inertia["delta"]
  print('Optimizacion finalizada.')
  return poblacion,Gbest,GbestHist

In [3]:
def poblation_creation(VariableLimits,populationNumber):
  casos=list()
  for variable in  VariableLimits:
    elemento=list(np.linspace(variable["lower"],variable["upper"],int(populationNumber**(1/len( VariableLimits)))))
    casos.append(elemento)

  Combinations=itertools.product(*casos)
  
  #Poblacion=[ {"position"=[3,4],"speed"=[]}]
  Poblacion=list()
  for case in Combinations:
    Poblacion.append({"position":list(case),"speed":list( np.random.uniform(size=len(case)))})
  return Poblacion

In [7]:
VariableLimits=[{"name":"x","upper":5,"lower":-5},{"name":"y","upper":5,"lower":-5}]
Pobla_uniforme=poblation_creation(VariableLimits,25)
# Pobla_uniforme

In [9]:
def prob_elipse(Parametros):
  ''' modelo de la elipse'''
  #definición de los objetivos
  x=Symbol('x')
  y=Symbol('y')

  A=x**2+y**2
  objetivo=A.subs([[x,Parametros[0]],[y,Parametros[1]]])
  
  return objetivo

In [12]:
# Poblacion,mejor=pso(prob_elipse,Pobla_uniforme,CogConst=1,SocialConst=1,Inertia={"min":0 ,"max":1,"delta":0.95,"gen":10},iterations=100)

In [13]:
# print('Mejor solucion:',mejor)

Mejor solucion: {'value': 6.49469971868384e-119, 'position': [7.850660886613553e-60, 1.8204726941743753e-60]}
